In [1]:
# imports

import sys
import pandas as pd
from datasets import load_from_disk
from scipy.stats import zscore
from sklearn.preprocessing import Normalizer
import anndata as ad
import scanpy as sc

sys.path.append('/work/magroup/kaileyhu/Geneformer')


print("imports done")

imports done


In [2]:
omics_expressions = pd.read_csv("/work/magroup/kaileyhu/datasets/depmap/OmicsExpressionProteinCodingGenesTPMLogp1.csv")
damaging_mat = pd.read_csv("/work/magroup/kaileyhu/datasets/depmap/matrices/OmicsSomaticMutationsMatrixDamaging.csv")
hotspot_mat = pd.read_csv("/work/magroup/kaileyhu/datasets/depmap/matrices/OmicsSomaticMutationsMatrixHotspot.csv")
vitality = pd.read_csv("/work/magroup/kaileyhu/datasets/depmap/CRISPRGeneEffect.csv")

In [3]:
cell_list = []
omics_expressions_cells = omics_expressions["Unnamed: 0"]
damaging_mat_cells = damaging_mat["Unnamed: 0"]
hotspot_mat_cells = hotspot_mat["Unnamed: 0"]
vitality_cells = vitality["Unnamed: 0"]

for cell in vitality_cells:
    if omics_expressions_cells.isin([cell]).any() and damaging_mat_cells.isin([cell]).any() and hotspot_mat_cells.isin([cell]).any():
        cell_list.append(cell)

In [4]:
omics_expressions_intersection = omics_expressions[(omics_expressions_cells.isin(cell_list))]
damaging_mat_intersection = damaging_mat[(damaging_mat_cells.isin(cell_list))]
hotspot_mat_intersection = hotspot_mat[(hotspot_mat_cells.isin(cell_list))]
vitality_intersection = vitality[(vitality_cells.isin(cell_list))]

In [5]:
adata = ad.AnnData(omics_expressions_intersection.loc[:, omics_expressions_intersection.columns != 'Unnamed: 0'])

adata.obs_names = [str(i) for i in omics_expressions_intersection.iloc[:, 0]]
adata.var_names = [str(i) for i in omics_expressions_intersection.columns[1:]]
print(adata.var_names)
print(adata.obs_names)
adata.X


Index(['TSPAN6 (7105)', 'TNMD (64102)', 'DPM1 (8813)', 'SCYL3 (57147)',
       'C1orf112 (55732)', 'FGR (2268)', 'CFH (3075)', 'FUCA2 (2519)',
       'GCLC (2729)', 'NFYA (4800)',
       ...
       'H3C2 (8358)', 'H3C3 (8352)', 'AC098582.1 (8916)',
       'DUS4L-BCAP29 (115253422)', 'C8orf44-SGK3 (100533105)',
       'ELOA3B (728929)', 'NPBWR1 (2831)', 'ELOA3D (100506888)',
       'ELOA3 (162699)', 'CDR1 (1038)'],
      dtype='object', length=19193)
Index(['ACH-001289', 'ACH-001538', 'ACH-000233', 'ACH-000461', 'ACH-001794',
       'ACH-002023', 'ACH-000528', 'ACH-001655', 'ACH-000792', 'ACH-001098',
       ...
       'ACH-000968', 'ACH-000114', 'ACH-000402', 'ACH-000036', 'ACH-000973',
       'ACH-001128', 'ACH-002669', 'ACH-001858', 'ACH-001997', 'ACH-000052'],
      dtype='object', length=1021)


/home/kaileyhu/anaconda3/envs/geneformer/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


array([[4.56742376, 0.5849625 , 7.10664145, ..., 0.        , 0.        ,
        0.        ],
       [5.08533967, 0.        , 7.15421053, ..., 0.        , 0.        ,
        0.        ],
       [0.05658353, 0.        , 6.09423607, ..., 0.05658353, 0.        ,
        0.        ],
       ...,
       [4.39094277, 0.        , 7.01323892, ..., 0.        , 0.        ,
        0.52606881],
       [5.05745027, 0.        , 7.81519114, ..., 0.        , 0.        ,
        0.        ],
       [4.24944534, 0.        , 6.17572419, ..., 0.        , 0.17632277,
        0.05658353]])

In [6]:
hotspot_mat_intersection.set_index('Unnamed: 0', inplace = True)
omics_expressions_intersection.set_index('Unnamed: 0', inplace = True)

In [7]:
vitality_intersection.set_index('Unnamed: 0', inplace = True)
# print(adata.obs_names)

In [10]:
adata.var_names[0]

'TSPAN6 (7105)'

In [11]:
adata.obs = hotspot_mat_intersection
# adata.obsm['spatial'] = adata.obs.loc[:,["TSPAN6 (7105)", "TNMD (64102)"]].values
# sc.pl.spatial(adata, spot_size=20)

In [12]:
adata.obs.columns

Index(['PIK3CD (5293)', 'MTOR (2475)', 'KLHDC7A (127707)', 'ARID1A (8289)',
       'ZSCAN20 (7579)', 'CSMD2 (114784)', 'EIF2B3 (8891)', 'HPDL (84842)',
       'STIL (6491)', 'C1orf141 (400757)',
       ...
       'AR (367)', 'MED12 (9968)', 'ATRX (546)', 'TRPC5 (7224)',
       'AMOT (154796)', 'DCAF12L2 (340578)', 'GPC3 (2719)', 'MAGEC1 (9947)',
       'GABRE (2564)', 'AGAP5 (729092)'],
      dtype='object', length=525)

In [13]:
adata.write_h5ad("/work/magroup/kaileyhu/datasets/depmap/processed/adata.h5ad",compression='gzip')

In [60]:
vitality

,Unnamed: 0,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,-0.122637,0.025881,0.034217,-0.128082,-0.031285,0.338046,-0.006439,-0.093642,0.189186,...,-0.131727,-0.039829,0.179405,0.283552,0.204513,-0.289724,-0.062972,0.074180,0.111244,-0.467908
1,ACH-000004,0.019756,-0.083640,-0.060118,-0.027417,-0.036116,-0.001056,0.312876,-0.086897,0.204434,...,-0.170329,-0.454263,0.194583,0.098989,0.126948,0.032983,-0.410392,0.113156,0.234388,-0.088306
2,ACH-000005,-0.107208,-0.023211,0.200204,0.116039,-0.172227,-0.071294,0.203270,-0.127806,-0.090981,...,-0.301695,-0.454969,-0.061959,-0.036427,0.016602,-0.201273,-0.178877,-0.055349,-0.002161,-0.186842
3,ACH-000007,-0.031027,-0.137850,0.067704,0.107988,0.007992,0.124945,0.049548,-0.220824,-0.165669,...,-0.303390,-0.507272,-0.025400,0.236659,0.072010,-0.100344,-0.462160,-0.001555,-0.325964,-0.486660
4,ACH-000009,0.008888,-0.146566,0.084471,0.089419,0.065109,0.027841,0.087943,-0.161369,0.041121,...,-0.255466,-0.288739,-0.037132,0.261444,-0.062391,-0.112703,-0.598698,0.095877,-0.026742,-0.320759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,ACH-002834,-0.055721,-0.121228,0.072790,0.021916,-0.221940,0.071135,0.134854,-0.504790,0.058356,...,-0.104959,-0.559438,-0.062461,-0.058814,-0.076221,0.024959,0.015259,-0.306562,-0.148368,-0.579576
1096,ACH-002847,-0.009973,-0.119813,0.044097,0.041358,-0.146565,0.160566,0.134247,-0.242759,-0.085035,...,-0.314751,-0.903238,0.104278,0.101270,0.026373,-0.119911,-0.289412,-0.195097,-0.206400,-0.486525
1097,ACH-002922,-0.025991,-0.007706,-0.038468,0.236576,-0.239690,0.061611,0.128461,-0.498119,0.022097,...,-0.164150,-0.743401,-0.023401,0.174912,-0.230089,-0.079342,-0.347484,-0.085302,-0.095965,-0.346272
1098,ACH-002925,-0.127639,-0.040705,0.134556,-0.047984,-0.116114,0.213144,0.198647,-0.183355,-0.004409,...,-0.275523,-0.898982,-0.050201,0.125480,-0.189114,-0.043555,-0.335270,-0.208063,-0.094741,-0.222404


In [88]:
expr_genes = [i for i in omics_expressions_intersection.columns]
damaging_genes = [i for i in damaging_mat_intersection.columns]
hotspot_genes = [i for i in hotspot_mat_intersection.columns]
vitality_genes = [i for i in vitality_intersection.columns]

In [89]:
gene_list = []
for gene in expr_genes:
    # print(gene)
    if gene in damaging_genes and gene in hotspot_genes and gene in vitality_genes:
        gene_list.append(gene)

In [90]:
len(gene_list)

499

In [87]:
for gene in hotspot_genes:
    # print(gene)
    if gene not in damaging_genes:
        print(gene)

RIT1 (6016)
RRAS2 (22800)
SDHAF2 (54949)
ACTR10 (55860)
NQO1 (1728)
BCL2 (596)
FAM174C (55009)
ZNF439 (90594)
RALB (5899)
U2AF1 (7307)
H2BC9 (8345)
MYC (4609)
ZNF510 (22869)
AGAP5 (729092)


In [106]:
expr_genes_intersection = omics_expressions_intersection.loc[:, gene_list]
damaging_genes_intersection = damaging_mat_intersection.loc[:, gene_list]
hotspot_genes_intersection = hotspot_mat_intersection.loc[:, gene_list]
vitality_genes_intersection = vitality_intersection.loc[:, gene_list]

In [107]:
# expr_genes_intersection

,RAD52 (5893),CREBBP (1387),PON1 (5444),PKD1 (5310),NCAPD2 (9918),ANOS1 (3730),RABGAP1 (23637),DCN (1634),BRCA1 (672),TACC3 (10460),...,OR2A5 (393046),CEBPA (1050),ZNF564 (163050),HS3ST5 (222537),PCDHA12 (56137),PABPC4L (132430),ZNF432 (9668),GAS2L2 (246176),SRCIN1 (80725),H3C2 (8358)
1,2.969012,3.606442,0.014355,3.333424,5.736064,0.400538,5.269781,0.263034,3.810443,6.091488,...,0.000000,1.827819,2.201634,0.137504,0.084064,0.298658,2.782409,0.028569,1.226509,1.286881
3,2.689299,2.801159,0.000000,2.664483,6.531693,0.847997,4.557042,0.056584,3.430285,4.765004,...,0.000000,2.353323,2.778209,0.000000,0.000000,0.000000,2.185867,0.137504,0.863938,0.214125
7,4.076388,4.778734,0.070389,5.569248,6.350674,0.000000,4.292045,0.823749,4.683696,5.853996,...,0.124328,0.056584,2.035624,0.014355,0.000000,0.000000,0.070389,0.000000,2.408712,0.879706
8,2.996389,4.440952,0.000000,4.797532,6.692790,0.000000,5.153805,0.263034,4.747924,6.490410,...,0.000000,1.989139,1.505891,0.000000,1.531069,0.000000,2.053111,0.028569,2.543496,0.903038
10,2.568032,3.747387,0.056584,5.596637,4.993674,0.097611,3.683696,12.284477,4.228049,5.229203,...,0.000000,0.400538,1.981853,1.137504,0.014355,0.275007,2.283922,0.000000,0.042644,0.526069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,4.018812,4.077243,0.000000,5.743353,6.271650,0.536053,4.397118,0.782409,4.838952,6.594250,...,0.000000,1.803227,2.604071,0.000000,0.669027,0.000000,3.617063,0.028569,1.304511,0.475085
1475,2.829850,2.811471,0.000000,2.080658,6.434962,1.356144,4.969933,0.000000,3.714795,5.143230,...,0.000000,1.207893,2.568032,0.000000,0.275007,0.000000,2.713696,0.042644,0.918386,0.189034
1476,2.039138,3.130931,0.189034,3.629939,5.934045,0.378512,4.505256,0.226509,3.655352,6.616769,...,0.000000,0.028569,1.831877,0.042644,0.963474,0.226509,1.448901,0.000000,0.137504,1.097611
1477,2.643856,3.353323,0.432959,3.301588,5.645586,0.000000,4.444932,0.748461,4.201634,6.440122,...,0.000000,0.070389,2.592158,0.263034,0.000000,0.000000,2.073820,0.042644,0.356144,0.831877


In [110]:
adata_genes = ad.AnnData(expr_genes_intersection.loc[:, expr_genes_intersection.columns != 'Unnamed: 0'])

adata_genes.obs_names = [i for i in expr_genes_intersection.iloc[:, 0]]
adata_genes.var_names = [i for i in expr_genes_intersection.columns]
print(adata_genes.var_names)
print(adata_genes.obs_names)
adata_genes.X


Index(['RAD52 (5893)', 'CREBBP (1387)', 'PON1 (5444)', 'PKD1 (5310)',
       'NCAPD2 (9918)', 'ANOS1 (3730)', 'RABGAP1 (23637)', 'DCN (1634)',
       'BRCA1 (672)', 'TACC3 (10460)',
       ...
       'OR2A5 (393046)', 'CEBPA (1050)', 'ZNF564 (163050)', 'HS3ST5 (222537)',
       'PCDHA12 (56137)', 'PABPC4L (132430)', 'ZNF432 (9668)',
       'GAS2L2 (246176)', 'SRCIN1 (80725)', 'H3C2 (8358)'],
      dtype='object', length=499)
Index([ 2.969012307516316,  2.689299160535892,   4.07638806851284,
        2.996388746447621,  2.568032104771279, 2.5310694927259543,
        2.837943241891028, 2.3132458517875616,   3.37851162325373,
        2.875780063068488,
       ...
       2.7463127664254587,   3.93451650158608,  3.927896453728821,
        2.381283372503784,  4.222650022451478,  4.018812185742817,
       2.8298495598446904,  2.039138393906958,  2.643856189774725,
        3.981852653289741],
      dtype='float64', length=1021)


/home/kaileyhu/anaconda3/envs/geneformer/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/kaileyhu/anaconda3/envs/geneformer/lib/python3.9/site-packages/anndata/_core/anndata.py:933: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [2.969012307516316, 2.689299160535892, 4.07638806851284, 2.996388746447621, 2.568032104771279]

    Inferred to be: floating

  names = self._prep_dim_index(names, "obs")


array([[2.96901231, 3.60644223, 0.01435529, ..., 0.02856915, 1.22650853,
        1.28688115],
       [2.68929916, 2.80115866, 0.        , ..., 0.13750352, 0.86393845,
        0.21412481],
       [4.07638807, 4.77873424, 0.07038933, ..., 0.        , 2.40871186,
        0.87970577],
       ...,
       [2.03913839, 3.13093087, 0.18903382, ..., 0.        , 0.13750352,
        1.0976108 ],
       [2.64385619, 3.35332329, 0.43295941, ..., 0.04264434, 0.35614381,
        0.83187724],
       [3.98185265, 4.63691458, 0.02856915, ..., 0.01435529, 0.67807191,
        1.24488706]])

In [111]:
adata_genes.obs = hotspot_genes_intersection

In [ ]:
print(adata_genes.obs)

In [113]:
adata_genes.write_h5ad("/work/magroup/kaileyhu/datasets/depmap/processed/adata_genes.h5ad",compression='gzip')

In [15]:
# SynLethDB starting from here
SL = pd.read_csv("/work/magroup/kaileyhu/datasets/SynLethDB/Human_SL.csv")
non_SL = pd.read_csv("/work/magroup/kaileyhu/datasets/SynLethDB/Human_nonSL.csv")

In [16]:
cell_lines = []
for i in SL["r.cell_line"]:
    i = str(i).split(";")
    for j in i:
        j = j.lower()
        if j not in cell_lines and j != 'nan':
            cell_lines.append(j)

for i in non_SL["r.cell_line"]:
    i = str(i).split(";")
    for j in i:
        j = j.lower()
        if j not in cell_lines and j != 'nan':
            cell_lines.append(j)

len(cell_lines)

307

In [17]:
with open('/work/magroup/kaileyhu/datasets/SynLethDB/SL_cell_lines.txt', 'w') as f:
    for line in cell_lines:
        f.write(f"{line}\n")

In [18]:
metadata = pd.read_csv('/work/magroup/kaileyhu/datasets/depmap/metadata.csv')
metadata

,ModelID,PatientID,CellLineName,StrippedCellLineName,DepmapModelType,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,LegacyMolecularSubtype,...,TissueOrigin,CCLEName,CatalogNumber,PlateCoating,ModelDerivationMaterial,PublicComments,WTSIMasterCellID,SangerModelID,COSMICID,LegacySubSubtype
0,ACH-000001,PT-gj46wT,NIH:OVCAR-3,NIHOVCAR3,HGSOC,Ovary/Fallopian Tube,Ovarian Epithelial Tumor,High-Grade Serous Ovarian Cancer,HGSOC,NaN,...,NaN,NIHOVCAR3_OVARY,HTB-71,NaN,NaN,NaN,2201.0,SIDM00105,905933.0,high_grade_serous
1,ACH-000002,PT-5qa3uk,HL-60,HL60,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,NaN,...,NaN,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,CCL-240,NaN,NaN,NaN,55.0,SIDM00829,905938.0,M3
2,ACH-000003,PT-puKIyc,CACO2,CACO2,COAD,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,NaN,...,NaN,CACO2_LARGE_INTESTINE,HTB-37,NaN,NaN,NaN,NaN,SIDM00891,NaN,NaN
3,ACH-000004,PT-q4K2cp,HEL,HEL,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,NaN,...,NaN,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACC 11,NaN,NaN,NaN,783.0,SIDM00594,907053.0,M6
4,ACH-000005,PT-q4K2cp,HEL 92.1.7,HEL9217,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,NaN,...,NaN,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HEL9217,NaN,NaN,NaN,NaN,SIDM00593,NaN,M6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,ACH-003157,PT-QDEP9D,ABM-T0822,ABMT0822,ZIMMMPLC,Lung,Non-Cancerous,Immortalized MPLC Cells,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1917,ACH-003158,PT-nszsxG,ABM-T9220,ABMT9220,ZIMMSMCI,Muscle,Non-Cancerous,"Immortalized Smooth Muscle Cells, Intestinal",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1918,ACH-003159,PT-AUxVvV,ABM-T9233,ABMT9233,ZIMMRSCH,Hair,Non-Cancerous,Immortalized Hair Follicle Inner Root Sheath C...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919,ACH-003160,PT-AUxVvV,ABM-T9249,ABMT9249,ZIMMGMCH,Hair,Non-Cancerous,Immortalized Hair Germinal Matrix Cells,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
metadata_lines = []
for i in metadata["StrippedCellLineName"]:
    i = str(i).split(";")
    for j in i:
        j = j.lower()
        if j not in metadata_lines and j != 'nan':
            metadata_lines.append(j)

In [36]:
overlap = []
for i in cell_lines:
    if i in metadata_lines:
        overlap.append(i)
print(overlap)

['a375', 'k562', 'hela', 'hct116', 'a549', 'jurkat', '786o', 'ovcar8', 'meljuso', 'ht29']
